### Постановка задачи
Подготовим данные для построения модели: получим, объединим, оптимизируем и обогатим данные.

Сохраним готовые данные в нескольких форматах: CSV, HDF5

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Загрузка данных, отсечение 20 зданий, объединение и оптимизация

In [1]:
# Отключим предупреждения
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import os


def reduce_mem_usage(df: pd.DataFrame):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(
        'Потребление памяти меньше на ',
        round(start_mem - end_mem, 2),
        ' Мб (-',
        round(100 * (start_mem - end_mem) / start_mem, 1),
        '%)',
        sep=''
    )
    return df


buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

weather = weather[weather['site_id'] == 0]
energy = energy[energy['building_id'] < 20]

energy = pd.merge(left=energy, right=buildings, how='left', left_on='building_id', right_on='building_id')
del buildings

### Интерполяция значений

In [2]:
weather['precip_depth_1_hr'] = weather['precip_depth_1_hr'].map(lambda x: x if x > 0 else 0)
interpolate_columns = ['air_temperature', 'dew_temperature', 'cloud_coverage', 'wind_speed', 'wind_direction', 'precip_depth_1_hr', 'sea_level_pressure']
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both', kind='cubic')

### Обогащение данных: погода

In [3]:
weather['air_temperature_diff1'] = weather['air_temperature'].diff()
weather.at[0, 'air_temperature_diff1'] = weather.at[1, 'air_temperature_diff1']
weather['air_temperature_diff2'] = weather['air_temperature_diff1'].diff()
weather.at[0, 'air_temperature_diff2'] = weather.at[1, 'air_temperature_diff2']

### Объединение погодных данных

In [4]:
energy = pd.merge(
    left=energy.set_index(['timestamp', 'site_id']), 
    right=weather.set_index(['timestamp', 'site_id']),
    how='left', left_index=True, right_index=True
)
energy.reset_index(inplace=True)
del weather
energy = reduce_mem_usage(energy)
energy.info()

Потребление памяти меньше на 16.42 Мб (-68.1%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   site_id                175680 non-null  int8          
 2   building_id            175680 non-null  int8          
 3   meter                  175680 non-null  int8          
 4   meter_reading          175680 non-null  float16       
 5   primary_use            175680 non-null  category      
 6   square_feet            175680 non-null  int32         
 7   year_built             175680 non-null  float16       
 8   floor_count            0 non-null       float64       
 9   air_temperature        175680 non-null  float16       
 10  cloud_coverage         175680 non-null  float16       
 11  dew_temperature        175680 non-null  float16       
 1

### Обогащение данных: дата

In [5]:
energy['hour'] = energy['timestamp'].dt.hour.astype('int8')
energy['weekday'] = energy['timestamp'].dt.weekday.astype('int8')
energy['week'] = energy['timestamp'].dt.isocalendar().week.astype('int8')
energy['month'] = energy['timestamp'].dt.month.astype('int8')
energy['date'] = pd.to_datetime(energy['timestamp'].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype('int8')

for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype('int8')
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype('int8')
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype('int8')

### Логарифмирование данных
$z = A * x + B * y -> log z = A * x + B * y => z = e^{Ax} * e^{By} => z = a^x * b^y$

In [6]:
energy['meter_reading_log'] = np.log(energy['meter_reading'] + 1)

### Экспорт данных в CSV

In [7]:
print(energy.info())
energy.to_csv('energy.0-20.ready.csv.gz', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 97 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   site_id                175680 non-null  int8          
 2   building_id            175680 non-null  int8          
 3   meter                  175680 non-null  int8          
 4   meter_reading          175680 non-null  float16       
 5   primary_use            175680 non-null  category      
 6   square_feet            175680 non-null  int32         
 7   year_built             175680 non-null  float16       
 8   floor_count            0 non-null       float64       
 9   air_temperature        175680 non-null  float16       
 10  cloud_coverage         175680 non-null  float16       
 11  dew_temperature        175680 non-null  float16       
 12  precip_depth_1_hr      175680 non-null  floa

In [8]:
energy = pd.read_csv('energy.0-20.ready.csv.gz')
print(energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 97 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   timestamp              175680 non-null  object 
 1   site_id                175680 non-null  int64  
 2   building_id            175680 non-null  int64  
 3   meter                  175680 non-null  int64  
 4   meter_reading          175680 non-null  float64
 5   primary_use            175680 non-null  object 
 6   square_feet            175680 non-null  int64  
 7   year_built             175680 non-null  float64
 8   floor_count            0 non-null       float64
 9   air_temperature        175680 non-null  float64
 10  cloud_coverage         175680 non-null  float64
 11  dew_temperature        175680 non-null  float64
 12  precip_depth_1_hr      175680 non-null  float64
 13  sea_level_pressure     175680 non-null  float64
 14  wind_direction         175680 non-nu

### Экспорт данных в HDF5
HDF5: / ->
* Группа (+ метаданные)
 * Набор данных
 * ...

In [9]:
energy = reduce_mem_usage(energy)
energy.to_hdf('energy.0-20.ready.h5', 'energy', format='table', complevel=9, mode='w')
print('CSV:', os.path.getsize(os.getcwd() + '\energy.0-20.ready.csv.gz'))
print('HDF5:', os.path.getsize(os.getcwd() + '\energy.0-20.ready.h5'))

Потребление памяти меньше на 108.72 Мб (-83.6%)
CSV: 1803835
HDF5: 899154


In [10]:
energy = pd.read_hdf('energy.0-20.ready.h5', 'energy')
energy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 97 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   site_id                175680 non-null  int8          
 2   building_id            175680 non-null  int8          
 3   meter                  175680 non-null  int8          
 4   meter_reading          175680 non-null  float16       
 5   primary_use            175680 non-null  category      
 6   square_feet            175680 non-null  int32         
 7   year_built             175680 non-null  float16       
 8   floor_count            0 non-null       float64       
 9   air_temperature        175680 non-null  float16       
 10  cloud_coverage         175680 non-null  float16       
 11  dew_temperature        175680 non-null  float16       
 12  precip_depth_1_hr      175680 non-null  floa

In [11]:
energy_train, energy_test = train_test_split(energy[energy['meter_reading'] > 0], test_size=.2)
energy_train.head()

,timestamp,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,floor_count,air_temperature,...,is_m4,is_m5,is_m6,is_m7,is_m8,is_m9,is_m10,is_m11,is_m12,meter_reading_log
99751,2016-07-26 19:00:00,0,11,0,480.50,Education,49073,1968.0,NaN,33.906250,...,0,0,0,1,0,0,0,0,0,6.175781
104643,2016-08-06 00:00:00,0,3,0,364.25,Education,23685,2002.0,NaN,26.703125,...,0,0,0,0,1,0,0,0,0,5.902344
152175,2016-11-13 00:00:00,0,15,0,194.25,Office,83957,1974.0,NaN,21.093750,...,0,0,0,0,0,0,0,1,0,5.273438
82994,2016-06-21 21:00:00,0,14,0,435.50,Education,86250,2013.0,NaN,30.000000,...,0,0,1,0,0,0,0,0,0,6.078125
143620,2016-10-26 05:00:00,0,0,0,220.50,Education,7432,2008.0,NaN,18.906250,...,0,0,0,0,0,0,1,0,0,5.402344


In [12]:
pd.set_option('io.hdf.default_format', 'table')
store = pd.HDFStore('energy.0-20.ready.split.h5', complevel=9, complib='zlib', mode='w')
store['energy_train'] = energy_train
store['energy_test'] = energy_test
store.put('metadata', pd.Series(['Набор обогащенных тестовых данных по 20 зданиям']))
store.close()
print('HDF5:', os.path.getsize(os.getcwd() + '\energy.0-20.ready.split.h5'))

HDF5: 4120641


Для хранения атрибутов наборов данных также можно использовать

store.get_storer('energy_train').attrs.my_attr

### Чтение из HDF5

In [13]:
store = pd.HDFStore('energy.0-20.ready.split.h5')
energy_test = store.get('energy_test')[:]
energy_train = store.get('energy_train')[:]
metadata = store.get('metadata')[:]
store.close()
print(metadata[0])
energy_train.head()

Набор обогащенных тестовых данных по 20 зданиям


,timestamp,site_id,building_id,meter,meter_reading,primary_use,square_feet,year_built,floor_count,air_temperature,...,is_m4,is_m5,is_m6,is_m7,is_m8,is_m9,is_m10,is_m11,is_m12,meter_reading_log
99751,2016-07-26 19:00:00,0,11,0,480.50,Education,49073,1968.0,NaN,33.906250,...,0,0,0,1,0,0,0,0,0,6.175781
104643,2016-08-06 00:00:00,0,3,0,364.25,Education,23685,2002.0,NaN,26.703125,...,0,0,0,0,1,0,0,0,0,5.902344
152175,2016-11-13 00:00:00,0,15,0,194.25,Office,83957,1974.0,NaN,21.093750,...,0,0,0,0,0,0,0,1,0,5.273438
82994,2016-06-21 21:00:00,0,14,0,435.50,Education,86250,2013.0,NaN,30.000000,...,0,0,1,0,0,0,0,0,0,6.078125
143620,2016-10-26 05:00:00,0,0,0,220.50,Education,7432,2008.0,NaN,18.906250,...,0,0,0,0,0,0,1,0,0,5.402344
